## Linear-in-Means (2-parameter) model

**Model.** We specialize the structural mapping to a linear-in-means form with **no self-loops** and **two parameters** $\theta=(\rho,\beta)$:
$$
Y \;=\; (I - \rho P)^{-1}\,\big(X\beta + \varepsilon\big),
$$
where $P$ is a peer operator with zero diagonal (optionally row-normalized), $X\in\mathbb{R}^{n\times 1}$ is a single covariate (scalar loading $\beta$ keeps the model two-parameter), and $\varepsilon$ is mean-zero noise with **fixed variance** (not estimated).

**No self-loops.** We take
$$
P \;=\; A - \mathrm{diag}(A),
$$
optionally followed by row (degree) normalization. Only $\mathrm{diag}(P)=0$ is required.

**Invertibility / stability.** We guard against singularity of $I-\rho P$ by enforcing
$$
|\rho| \;<\; \frac{1}{\lambda_{\max}(P)}\,,
$$
with $\lambda_{\max}(P)$ the spectral radius. (For row-normalized $P$, typically $\lambda_{\max}\le 1$.)

**Why “2-parameter”?** The noise scale (e.g. $\varepsilon\sim\mathcal{N}(0,\sigma^2 I)$) is **fixed** and not estimated, so the only free structural parameters are the peer effect $\rho$ and the scalar loading $\beta$.

**Adversarial estimation hook.** The generator returns $Y'(\rho,\beta)$ for fixed $(X,P)$. Your adversarial loop (same subgraph sampler and discriminator) can keep using cross-entropy on real vs synthetic subgraphs as the outer loss while the optimizer searches over $(\rho,\beta)$.


## Imports

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="An issue occurred while importing 'torch-sparse'")
warnings.filterwarnings("ignore", message="An issue occurred while importing 'torch-cluster'")

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

from adversarial_nets import (
    AdversarialEstimator,
    GraphDataset,
    objective_function
)

## Test dataset 

In [ ]:
N_NODES = 1000
N_SAMPLES = 800
RESOLUTION = 10
TRUE_PARAMS = [0.5, 0.7]


def create_test_graph_dataset(
    num_nodes: int = 100,
    true_a: float = 0.2,     # peer effect α; must satisfy |α| < 1/ρ(P)
    true_b: float = 2.0,     # loading β on X_1
    p: float = 0.01,
    seed: int = 42,
    noise_scale: float = 0.01,
    row_normalize: bool = True,
) -> GraphDataset:
    """Generate a test graph dataset for a linear-in-means model:
       Y = (I - α P)^{-1}(β X_1 + ε)
    """
    rng = np.random.default_rng(seed)

    G = nx.erdos_renyi_graph(n=num_nodes, p=p, seed=seed)
    A = nx.to_numpy_array(G, dtype=float)
    np.fill_diagonal(A, 0.0)

    if row_normalize:
        deg = A.sum(axis=1, keepdims=True)
        P = A / np.where(deg == 0.0, 1.0, deg)
    else:
        P = A

    X = rng.normal(0.0, 1.0, size=(num_nodes, 1))
    eps = rng.normal(0.0, noise_scale, size=num_nodes)

    eigvals = np.linalg.eigvals(P) if P.size else np.array([0.0])
    rad = float(np.max(np.abs(eigvals))) or 1.0
    if not (abs(true_a) < 1.0 / rad):
        raise ValueError(f"|alpha| must be < 1/rho(P) ≈ {1.0/rad:.3f}, got {true_a}")

    I = np.eye(num_nodes)
    rhs = true_b * X[:, 0] + eps
    Y = np.linalg.solve(I - true_a * P, rhs).reshape(-1, 1)

    N = list(range(num_nodes))
    return GraphDataset(X=X, Y=Y, A=A, N=N)


## Structural model mapping

In [ ]:
def linear_in_means_model(x, adjacency, y0, theta):
    """
    Two-parameter linear-in-means generator (signature preserved).

        Y = (I - alpha P)^{-1}(beta X_1 + eps)

    alpha: peer-effect parameter
    beta : scalar loading on the first covariate X_1
    P    : row-normalized peer operator with no self-loops
    eps  : mean-zero noise with FIXED variance

    Returns
    -------
    y : (n, 1) ndarray
    """

    if not hasattr(linear_in_means_model, "_rng"):
        linear_in_means_model._rng = np.random.default_rng(0)
    if not hasattr(linear_in_means_model, "_noise"):
        linear_in_means_model._noise = {}

    def _to_numpy(A):
        if hasattr(A, "toarray"):
            A = A.toarray()
        else:
            A = np.asarray(A)
        return A.astype(float, copy=False)

    def _build_peer_operator(A, row_normalize=True):
        P = A.astype(float, copy=True)
        np.fill_diagonal(P, 0.0)
        if row_normalize:
            deg = P.sum(axis=1, keepdims=True)
            with np.errstate(divide="ignore", invalid="ignore"):
                P = P / np.where(deg == 0.0, 1.0, deg)
                P[~np.isfinite(P)] = 0.0
        return P

    X = _to_numpy(x)
    A = _to_numpy(adjacency)
    n = A.shape[0]

    a, b = float(theta[0]), float(theta[1])
    P = _build_peer_operator(A, row_normalize=True)

    noise_scale = 0.01
    noise_cache = linear_in_means_model._noise
    if n not in noise_cache:
        noise_cache[n] = linear_in_means_model._rng.normal(0.0, noise_scale, size=n)
    eps = noise_cache[n]

    x1 = X if X.ndim == 1 else X[:, 0]
    rhs = b * x1 + eps

    I = np.eye(n)
    M = I - a * P
    try:
        y = np.linalg.solve(M, rhs)
    except np.linalg.LinAlgError:
        y = np.linalg.solve(M + 1e-8 * I, rhs)

    return y.reshape(-1, 1)


## Discriminator model


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SGConv
from torch_geometric.nn.aggr import AttentionalAggregation

def discriminator_factory(input_dim, hidden_dim=16, num_classes=2,
                          dropout_rate=0.1, K=1, add_self_loops=False):
    class TinySGC(nn.Module):
        def __init__(self, in_dim, hid_dim, num_cls):
            super().__init__()
            self.sgc = SGConv(in_channels=in_dim, out_channels=hid_dim,
                              K=K, cached=False, add_self_loops=add_self_loops, bias=True)
            self.pool = AttentionalAggregation(gate_nn=nn.Linear(hid_dim, 1, bias=False))
  
            self.head = nn.Linear(hid_dim, num_cls)

        def forward(self, data):
            x, edge_index, batch = data.x, data.edge_index, data.batch
            h = self.sgc(x, edge_index)
            h = F.dropout(h, p=dropout_rate, training=self.training)
            g = self.pool(h, batch)          
            return self.head(g)

    return TinySGC(input_dim, hidden_dim, num_classes)


## Visualization utils 

In [ ]:
def visualize_objective_surface(estimator, m, resolution, num_epochs, verbose=False):
    a_range = np.linspace(-5, 5, resolution)
    b_range = np.linspace(-5, 5, resolution)
    A, B = np.meshgrid(a_range, b_range)

    Z = np.zeros_like(A)

    total_evals = resolution * resolution
    with tqdm(total=total_evals, desc="Evaluating objective surface") as pbar:
        for i in range(resolution):
            for j in range(resolution):
                theta = [A[i, j], B[i, j]]
                Z[i, j] = objective_function(
                    theta,
                    estimator.ground_truth_generator,
                    estimator.synthetic_generator,
                    discriminator_factory=estimator.discriminator_factory,
                    num_epochs=num_epochs,
                    m=m,
                    verbose=verbose,
                    num_runs=12,
                    k_hops=1
                )
                pbar.update(1)

    fig = plt.figure(figsize=(12, 5))

    ax1 = fig.add_subplot(121, projection='3d')
    surf = ax1.plot_surface(A, B, Z, cmap='viridis', alpha=0.8)
    ax1.set_xlabel('Parameter a')
    ax1.set_ylabel('Parameter b')
    ax1.set_zlabel('Discriminator Accuracy')
    ax1.set_title('Objective Function Surface')

    true_a, true_b = TRUE_PARAMS[0], TRUE_PARAMS[1] 
    ax1.scatter([true_a], [true_b], [Z.min()], color='red', s=100, marker='*', label='True params')

    ax2 = fig.add_subplot(122)
    contour = ax2.contour(A, B, Z, levels=20, cmap='viridis')
    ax2.clabel(contour, inline=True, fontsize=8)
    ax2.scatter([true_a], [true_b], color='red', s=100, marker='*', label='True params')

    if hasattr(estimator, 'estimated_params') and estimator.estimated_params is not None:
        est_a, est_b = estimator.estimated_params
        ax1.scatter([est_a], [est_b], [Z.min()], color='orange', s=100, marker='^', label='Estimated params')
        ax2.scatter([est_a], [est_b], color='orange', s=100, marker='^', label='Estimated params')

    ax2.set_xlabel('Parameter a')
    ax2.set_ylabel('Parameter b')
    ax2.set_title('Objective Function Contours')
    ax2.legend()

    plt.colorbar(surf, ax=ax1, shrink=0.5)
    plt.tight_layout()
    plt.show()

    return Z, (A, B)

## Generate dataset


In [ ]:
print("Testing Adversarial Estimation for Linear-in-Means Model")
print("=" * 60)
print("\n1. Generating test dataset...")
test_data = create_test_graph_dataset(num_nodes=N_NODES, true_a=TRUE_PARAMS[0], true_b=TRUE_PARAMS[1], p=0.05, noise_scale=0.001)

## Create estimator


In [ ]:
print("\n2. Creating adversarial estimator...")
estimator = AdversarialEstimator(
    ground_truth_data=test_data,
    structural_model=linear_in_means_model,
    initial_params=[0.0, 0.0],
    bounds=[(0.0, 1.0), (0.0,1.0)],
    discriminator_factory=discriminator_factory,
    gp_params={
        'initial_point_generator': 'sobol',
        'n_initial_points': 100,
        'noise': 0.0001,
        'n_calls':150,
        'acq_func': 'PI'
    },
    metric="neg_logloss",
)

## Calibrate discriminator hyperparameters


In [ ]:
print("\n3. Calibrating discriminator hyperparameters...")
search_space = {
    'discriminator_params': {
        "hidden_dim":    lambda t: t.suggest_int("hidden_dim",6, 12, step=1),
        "dropout_rate":  lambda t: t.suggest_float("dropout_rate", 0.0, 0.20, step=0.01)
    },
    'training_params': {
        "lr":             lambda t: t.suggest_float("lr", 1e-4, 1e-2, step=0.0001),
        "weight_decay":   lambda t: t.suggest_float("weight_decay", 0.0, 1e-2, step=0.005),
        "label_smoothing":lambda t: t.suggest_float("label_smoothing", 0.0, 1e-2, step=0.005),
        "batch_size":     lambda t: t.suggest_categorical("batch_size", [64, 128, 256]),
        "num_epochs":     lambda t: t.suggest_int("num_epochs", 7, 8),
    }
}

optimizer_params = {'n_trials': 20}
estimator.callibrate(search_space, optimizer_params, metric_name='ace', k=5, num_runs=5, k_hops=1)

In [ ]:
estimator.calibration_study.best_params

In [ ]:
visualize_objective_surface(
    resolution=25, 
    estimator=estimator, 
    num_epochs=estimator.calibration_study.best_params['num_epochs'], 
    m=N_SAMPLES
    )

plt.show()

## Adversarial estimation


In [ ]:
print("4. Running adversarial estimation...")
result = estimator.estimate(
    m=N_SAMPLES,
    verbose=True,
    k_hops=1,
    num_runs=5
)

estimated_params = result['x'] if isinstance(result, dict) else result.x
estimator.estimated_params = estimated_params

print("5. Results:")
print(f"   - True parameters: alpha={TRUE_PARAMS[0]}, beta={TRUE_PARAMS[1]}")
print(f"   - Estimated parameters: alpha={estimated_params[0]:.4f}, beta={estimated_params[1]:.4f}")
print(
    f"   - Estimation error: alpha_error={abs(estimated_params[0] - TRUE_PARAMS[0]):.4f}, "
    f"beta_error={abs(estimated_params[1] - TRUE_PARAMS[1]):.4f}"
)

plt.tight_layout()
plt.show()